In [153]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

# Chrome WebDriver의 경로 설정
webdriver_path = "/path/to/chromedriver"

# Chrome WebDriver 옵션 설정
chrome_options = Options()
chrome_options.add_argument("--headless")  # 브라우저 창을 띄우지 않고 실행

# WebDriver 생성
driver = webdriver.Chrome(service=Service(webdriver_path), options=chrome_options)

# 웹 페이지 로드
url = "https://www.millie.co.kr/v3/bookDetail/179606121?anchor=book"
driver.get(url)

# 페이지 로드를 기다림 (예: 5초 기다림)
driver.implicitly_wait(5)

# 필요한 정보 추출
category_element = driver.find_element(By.XPATH, '//p[text()="카테고리"]/following-sibling::strong/a')
publisher_element = driver.find_element(By.XPATH, '//li[p="출판사" or p="제작사"]/strong/a')
paperback_publish_date_elements = driver.find_elements(By.XPATH, '//li[p="종이책 출간일"]/strong/a')
if paperback_publish_date_elements:
    paperback_publish_date_element = paperback_publish_date_elements[0]
else:
    electronic_publish_date_element = driver.find_element(By.XPATH, '//li[p="전자책 출간일"]/strong')
    paperback_publish_date_element = electronic_publish_date_element

# 책 제목과 작가 정보 추출
book_title_element = driver.find_element(By.XPATH, '//div[@class="meta-data"]/p[@class="book-name"]')
author_element = driver.find_element(By.XPATH, '//div[@class="meta-data"]/p[@class="author"]/span')

# 썸네일 이미지 URL 추출
thumbnail_element = driver.find_element(By.XPATH, '//div[@class="book-picture imageLoaded"]/picture/img')
thumbnail_url = thumbnail_element.get_attribute('src')

# 책 소개 추출
book_intro_element = driver.find_element(By.XPATH, './/div[@class="text-area desc more"]/p')
book_intro = book_intro_element.text.strip()

category = category_element.text
publisher = publisher_element.text
paperback_publish_date = paperback_publish_date_element.text
book_title = book_title_element.text.strip()
author = author_element.text.strip()

# 정보를 배열에 담기
book_info = [category, publisher, paperback_publish_date, book_title, author, thumbnail_url, book_intro]

# 출력
print("카테고리:", book_info[0])
print("종이책 출간일:", book_info[1])
print("출판사:", book_info[2])
print("책 제목:", book_info[3])
print("작가 정보:", book_info[4])
print("썸네일 이미지 URL:", book_info[5])
print("책 소개:", book_info[6])

reviews_data = []

reviews = driver.find_elements(By.CSS_SELECTOR, ".info")
for review in reviews:
    # 작성자 가져오기
    author_element = review.find_elements(By.CSS_SELECTOR, ".nickname a")
    if author_element:
        author = author_element[0].text
    else:
        continue

    # 작성일 가져오기
    date_element = review.find_elements(By.CSS_SELECTOR, ".date")
    if date_element:
        date = date_element[0].text
    else:
        continue

    # 리뷰 내용 가져오기
    content_element = review.find_elements(By.CSS_SELECTOR, ".cont")
    if content_element:
        content = content_element[0].text
    else:
        continue

    
    # 리뷰 데이터 저장
    review_data = {
        "작성자": author,
        "작성일": date,
        "리뷰 내용": content
    }
    reviews_data.append(review_data)

# WebDriver 종료
driver.quit()

# 리뷰 데이터 출력
for review_data in reviews_data:
    for key, value in review_data.items():
        print(key + ":", value)
    print()
    
import pymysql

DB_HOST = "iotsam.com"
DB_USER = "book"
DB_PASSWORD = "book"
DB_NAME = "book"
DB_PORT = 3306

# 데이터베이스 연결 함수
def connect_database():
    try:
        # MySQL 연결
        conn = pymysql.connect(
            host=DB_HOST,
            user=DB_USER,
            password=DB_PASSWORD,
            database=DB_NAME,
            port=DB_PORT
        )
        print("데이터베이스에 연결되었습니다.")
        return conn
    except pymysql.Error as error:
        print("데이터베이스 연결 오류: {}".format(error))
        return None

# 데이터베이스 연결
connection = connect_database()

# INSERT 쿼리 실행
if connection:
    try:
        with connection.cursor() as cursor:
            # 책 정보를 먼저 삽입한 후, 해당 idx 값을 가져옴
            sql = "INSERT INTO bookinfo (genre, publisher, publication_date, title, writer, poster, summary) VALUES (%s, %s, %s, %s, %s, %s, %s)"
            cursor.execute(sql, book_info)
            connection.commit()
            book_info_idx = cursor.lastrowid

            # feed 테이블에 데이터 삽입
            sql = "INSERT INTO feed (content, create_date, writer, bookinfo_idx) VALUES (%s, %s, %s, %s)"
            for review_data in reviews_data:
                content = review_data['리뷰 내용']
                create_date = review_data['작성일']
                writer = review_data['작성자']
                cursor.execute(sql, (content, create_date, writer, book_info_idx))
            connection.commit()

            print("데이터가 성공적으로 추가되었습니다.")
    except pymysql.Error as error:
        print("INSERT 오류: {}".format(error))
else:
    print("데이터베이스 연결이 실패했습니다.")

# 연결 종료
if connection:
    connection.close()
    print("데이터베이스 연결이 종료되었습니다.")



카테고리: 에세이
종이책 출간일: 위즈덤하우스
출판사: 2023.05.17
책 제목: 내가 너의 행복이 될게
작가 정보: 김미묘
썸네일 이미지 URL: https://cover.millie.co.kr/service/cover/179606121/84bbd8ce0ab447db8e33c025817d7e2a.jpg?w=220&q=80
책 소개: “충분히 잘해왔고 앞으로도 잘할 거야”
14만 팔로워가 기다려온 미묘와 친구들의 다정다감한 일상

혼자 남아 쓸쓸한 날, 다정한 위로가 필요한 날에 조그맣고 귀여운 친구들이 모여 앉아 내 이야기를 들어준다면. 동글동글한 캐릭터들이 나누는 다정한 대화들로 14만 팔로워의 사랑을 받아온 김미묘 작가의 첫 번째 그림 에세이 《내가 너의 행복이 될게》가 위즈덤하우스에서 출간되었다.
SNS에 만화를 연재하기 시작한 지 1년 반 만에 10만 명의 마음을 사로잡은 저자가 풀어냈던 이야기들을 엄선하고 미공개 에피소드, 솔직담백한 에세이, Q&A 등을 추가로 선보이며 미묘, 꾸꾸이, 행곰, 우비, 곤뇽이 다섯 캐릭터들 사이의 관계성이 돋보이는 다정다감한 김미묘 세계관에 깊이를 더한다. 미묘와 친구들이 매일매일의 고민과 행복 그리고 내일의 꿈에 대해 나누는 대화를 따라가다 보면 지친 우리에게 다정한 말 한마디가 얼마나 힘이 되는지 깨닫게 될 것이다.
작성자: 오!!
작성일: 2023.06.15
리뷰 내용: 쉴 때 편안하게 읽을 수 있었어요.

작성자: 코코88
작성일: 2023.06.09
리뷰 내용: 가볍게 읽기 좋은 힐링 책

작성자: Snow_love
작성일: 2023.06.02
리뷰 내용: 가벼운 마음으로 힐링 받고 갑니다 :)



In [189]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import pymysql

# Chrome WebDriver의 경로 설정
webdriver_path = "/path/to/chromedriver"

# Chrome WebDriver 옵션 설정
chrome_options = Options()
chrome_options.add_argument("--headless")  # 브라우저 창을 띄우지 않고 실행

# 데이터베이스 연결 설정
DB_HOST = "iotsam.com"
DB_USER = "book"
DB_PASSWORD = "book"
DB_NAME = "book"
DB_PORT = 3306

# 데이터베이스 연결 함수
def connect_database():
    try:
        # MySQL 연결
        conn = pymysql.connect(
            host=DB_HOST,
            user=DB_USER,
            password=DB_PASSWORD,
            database=DB_NAME,
            port=DB_PORT
        )
        print("데이터베이스에 연결되었습니다.")
        return conn
    except pymysql.Error as error:
        print("데이터베이스 연결 오류: {}".format(error))
        return None

# 책 정보 수집 및 데이터베이스 저장 함수
def scrape_book_info(url):
    # WebDriver 생성
    driver = webdriver.Chrome(service=Service(webdriver_path), options=chrome_options)

    # 웹 페이지 로드
    driver.get(url)

    # 페이지 로드를 기다림 (예: 5초 기다림)
    driver.implicitly_wait(5)

    # 필요한 정보 추출
    category_element = driver.find_element(By.XPATH, '//p[text()="카테고리"]/following-sibling::strong/a')
    publisher_element = driver.find_element(By.XPATH, '//li[p="출판사" or p="제작사"]/strong/a')
    paperback_publish_date_elements = driver.find_elements(By.XPATH, '//li[p="종이책 출간일"]/strong/a')
    if paperback_publish_date_elements:
        paperback_publish_date_element = paperback_publish_date_elements[0]
        paperback_publish_date = paperback_publish_date_element.text
    else:
        electronic_publish_date_element = driver.find_element(By.XPATH, '//li[p="전자책 출간일"]/strong/a')
        paperback_publish_date = electronic_publish_date_element.text

    # 책 제목과 작가 정보 추출
    book_title_element = driver.find_element(By.XPATH, '//div[@class="meta-data"]/p[@class="book-name"]')
    author_element = driver.find_element(By.XPATH, '//div[@class="meta-data"]/p[@class="author"]/span')

    # 썸네일 이미지 URL 추출
    thumbnail_element = driver.find_element(By.XPATH, '//div[@class="book-picture imageLoaded"]/picture/img')
    thumbnail_url = thumbnail_element.get_attribute('src')

    # 책 소개 추출
    book_intro_element = driver.find_element(By.XPATH, './/div[@class="text-area desc more"]/p')
    book_intro = book_intro_element.text.strip()

    category = category_element.text
    publisher = publisher_element.text
    paperback_publish_date = paperback_publish_date_element.text
    book_title = book_title_element.text.strip()
    author = author_element.text.strip()

    # 정보를 배열에 담기
    book_info = [category, publisher, paperback_publish_date, book_title, author, thumbnail_url, book_intro]
    

    reviews_data = []

    reviews = driver.find_elements(By.CSS_SELECTOR, ".info")
    for review in reviews:
        # 작성자 가져오기
        author_element = review.find_elements(By.CSS_SELECTOR, ".nickname a")
        if author_element:
            author = author_element[0].text
        else:
            continue

        # 작성일 가져오기
        date_element = review.find_elements(By.CSS_SELECTOR, ".date")
        if date_element:
            date = date_element[0].text
        else:
            continue

        # 리뷰 내용 가져오기
        content_element = review.find_elements(By.CSS_SELECTOR, ".cont")
        if content_element:
            content = content_element[0].text
        else:
            continue

        
        # 리뷰 데이터 저장
        review_data = {
            "작성자": author,
            "작성일": date,
            "리뷰 내용": content
        }
        reviews_data.append(review_data)

    # WebDriver 종료
    driver.quit()

    # 데이터베이스 연결
    connection = connect_database()

        # INSERT 쿼리 실행
    if connection:
        try:
            with connection.cursor() as cursor:
                # 책 정보를 먼저 삽입한 후, 해당 idx 값을 가져옴
                sql = "INSERT INTO bookinfo (genre, publisher, publication_date, title, writer, poster, summary) VALUES (%s, %s, %s, %s, %s, %s, %s)"
                cursor.execute(sql, book_info)
                connection.commit()
                book_info_idx = cursor.lastrowid

                # feed 테이블에 데이터 삽입
                sql = "INSERT INTO feed (content, create_date, writer, bookinfo_idx) VALUES (%s, %s, %s, %s)"
                for review_data in reviews_data:
                    content = review_data['리뷰 내용']
                    create_date = review_data['작성일']
                    writer = review_data['작성자']
                    cursor.execute(sql, (content, create_date, writer, book_info_idx))
                connection.commit()

                print("데이터가 성공적으로 추가되었습니다.")
        except pymysql.Error as error:
            print("INSERT 오류: {}".format(error))
    else:
        print("데이터베이스 연결이 실패했습니다.")

In [190]:
# 수집할 URL 리스트 생성
base_url = "https://www.millie.co.kr/v3/bookDetail/"
start_number = 179605013
end_number = 179605013

urls = [base_url + str(number) + "?anchor=book" for number in range(start_number, end_number + 1)]

# 반복문을 사용하여 데이터 수집 및 저장
for url in urls:
    scrape_book_info(url)


AttributeError: 'NoneType' object has no attribute 'text'